In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
for dirname, _, filenames in os.walk('.'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch_directml
import torch.nn as nn
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup

In [ ]:
# -----------------------------
# Load and preprocess data
# -----------------------------
# Use Kaggle paths when running on Kaggle
# MODEL_PATH = "/kaggle/input/xlm-roberta-base-offline/xlm_roberta_base_offline"
MODEL_PATH = "C:/Users/satra/Downloads/xlm_roberta_base_offline"

# trn = "/kaggle/input/jigsaw-agile-community-rules/train.csv"
# tst = "/kaggle/input/jigsaw-agile-community-rules/test.csv"
# trn = "C:/Users/satra/PycharmProjects/kaggle-jigsaw/train.csv"
# tst = "C:/Users/satra/PycharmProjects/kaggle-jigsaw/test.csv"
# df_trn = pd.read_csv(trn)
# df_trn = df_trn.sample(frac=.01, random_state=42).reset_index(drop=True)
# df_tst = pd.read_csv(tst)


def getText(value):
    return str(value) if pd.notna(value) else ''


def fill_empty_examples_pandas(df):
    """Fills NaN in example columns with empty strings and then propagates non-empty examples."""
    example_cols = ['positive_example_1', 'positive_example_2', 'negative_example_1', 'negative_example_2']

    # Ensure all example columns are strings and NaNs are empty strings
    for col in example_cols:
        df[col] = df[col].fillna('').astype(str)

    # Propagate non-empty examples to empty slots
    df['positive_example_1'] = df['positive_example_1'].mask(df['positive_example_1'] == '', df[
'positive_example_2'])
    df['positive_example_2'] = df['positive_example_2'].mask(df['positive_example_2'] == '', df[
'positive_example_1'])
    df['negative_example_1'] = df['negative_example_1'].mask(df['negative_example_1'] == '', df[
'negative_example_2'])
    df['negative_example_2'] = df['negative_example_2'].mask(df['negative_example_2'] == '', df[
'negative_example_1'])

    return df


def create_multi_input_row(row, text_to_classify_field_name):
    """
    Creates the dictionary of 7 text inputs for a single row.
    'text_to_classify_field_name' specifies which column (e.g., 'body', 'positive_example_1')
    should be used for the 'text_to_classify' input.
    """
    return {
        "text_to_classify": getText(row[text_to_classify_field_name]), # This is the dynamic part
        "rule": getText(row["rule"]),
        "subreddit": getText(row["subreddit"]),
        "pos1": getText(row['positive_example_1']),
        "pos2": getText(row['positive_example_2']),
        "neg1": getText(row['negative_example_1']),
        "neg2": getText(row['negative_example_2']),
    }


# trn_path = "/kaggle/input/jigsaw-agile-community-rules/train.csv"
# tst_path = "/kaggle/input/jigsaw-agile-community-rules/test.csv"
trn_path = "C:/Users/satra/PycharmProjects/kaggle-jigsaw/train.csv"
tst_path = "C:/Users/satra/PycharmProjects/kaggle-jigsaw/test.csv"
df_trn_orig = pd.read_csv(trn_path) # Keep original df_trn for KFold split
df_tst_orig = pd.read_csv(tst_path)

# Apply example filling to original dataframes (before creating 'inputs' column)
df_trn_orig = fill_empty_examples_pandas(df_trn_orig)
df_tst_orig = fill_empty_examples_pandas(df_tst_orig)

df_trn_orig["inputs"] = df_trn_orig.apply(lambda row: create_multi_input_row(row, "body"), axis=1)
df_tst_orig["inputs"] = df_tst_orig.apply(lambda row: create_multi_input_row(row, "body"), axis=1)

print(f"Original df_trn_orig shape: {df_trn_orig.shape}")
print(f"Original df_tst_orig shape: {df_tst_orig.shape}")


text_feature_cols = [
    'body',
    'rule',
    'subreddit',
    'positive_example_1',
    'positive_example_2',
    'negative_example_1',
    'negative_example_2'
]

print("--- Comprehensive NaN Inspection for All Text Feature Columns ---")

# Count NaNs for each text feature column
print("\n--- NaN Counts per Text Feature Column ---")
print(df_trn_orig[text_feature_cols].isnull().sum())

# Analyze rows with NaNs in 'body' (most critical)
print("\n--- Analysis for 'body' column NaNs ---")
body_nan_rows = df_trn_orig[df_trn_orig['body'].isnull()]
if not body_nan_rows.empty:
    print(f"Number of rows with NaN in 'body': {len(body_nan_rows)}")
    print("Rule violation distribution for rows with NaN in 'body':")
    print(body_nan_rows['rule_violation'].value_counts(normalize=True))
else:
    print("No NaN values found in 'body' column.")

# Analyze rows with NaNs in 'rule'
print("\n--- Analysis for 'rule' column NaNs ---")
rule_nan_rows = df_trn_orig[df_trn_orig['rule'].isnull()]
if not rule_nan_rows.empty:
    print(f"Number of rows with NaN in 'rule': {len(rule_nan_rows)}")
    print("Rule violation distribution for rows with NaN in 'rule':")
    print(rule_nan_rows['rule_violation'].value_counts(normalize=True))
else:
    print("No NaN values found in 'rule' column.")

# Analyze rows with NaNs in 'subreddit'
print("\n--- Analysis for 'subreddit' column NaNs ---")
subreddit_nan_rows = df_trn_orig[df_trn_orig['subreddit'].isnull()]
if not subreddit_nan_rows.empty:
    print(f"Number of rows with NaN in 'subreddit': {len(subreddit_nan_rows)}")
    print("Rule violation distribution for rows with NaN in 'subreddit':")
    print(subreddit_nan_rows['rule_violation'].value_counts(normalize=True))
else:
    print("No NaN values found in 'subreddit' column.")

# Analyze rows where ANY of the example columns are NaN
print("\n--- Analysis for Example Columns NaNs ---")
example_only_cols = [col for col in text_feature_cols if 'example' in col]
df_any_example_nan = df_trn_orig[df_trn_orig[example_only_cols].isnull().any(axis=1)]
if not df_any_example_nan.empty:
    print(f"Number of rows with NaN in ANY example column: {len(df_any_example_nan)}")
    print("Rule violation distribution for rows with NaN in ANY example column:")
    print(df_any_example_nan['rule_violation'].value_counts(normalize=True))
else:
    print("No NaN values found in any example column.")

# Overall rule_violation distribution (for comparison)
print(f"\n--- Overall rule_violation distribution: ---")
print(df_trn_orig['rule_violation'].value_counts(normalize=True))

In [ ]:
MAX_LEN = 128
N_EPOCHS = 8
k_folds = 5
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

# -----------------------------
# Dataset
# -----------------------------
class MultiInputDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=MAX_LEN, is_test=False):
        self.dataframe = dataframe # This dataframe already has the 'inputs' column
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.is_test = is_test

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        item = {}
        # Loop over the 7 fields that are keys in text_inputs_dict
        for field in ["text_to_classify", "rule", "subreddit", "pos1", "pos2", "neg1", "neg2"]:
            encoded = self.tokenizer(
                row[field], # Access from the dictionary
                truncation=True,
                padding='max_length',
                max_length=MAX_LEN,
                return_tensors="pt"
            )
            for key in encoded:
                item[f"{field}_{key}"] = encoded[key].squeeze(0)

        if not self.is_test:
            item["label"] = torch.tensor(row["rule_violation"], dtype=torch.float32)
        return item

# -----------------------------
# Model
# -----------------------------
class MultiInputBERT(nn.Module):
    def __init__(self, model_name='xlm-roberta-base'):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Sequential(
            nn.Linear(768 * 7, 256), # 7 inputs * 768 hidden size
            nn.ReLU(),
            nn.Linear(256, 1) # Output a single logit
        )

    def forward(self, inputs):
        cls_outputs = []
        # Loop over the 7 fields that are keys in the 'inputs' dictionary
        for field in ["text_to_classify", "rule", "subreddit", "pos1", "pos2", "neg1", "neg2"]:
            out = self.bert(
                input_ids=inputs[f"{field}_input_ids"],
                attention_mask=inputs[f"{field}_attention_mask"]
            )
            cls_outputs.append(out.last_hidden_state[:, 0])  # CLS token
        x = torch.cat(cls_outputs, dim=1)
        x = self.dropout(x)
        return self.classifier(x)

In [ ]:
# -----------------------------
# Training and Evaluation
# -----------------------------


def get_device():
    # Try to detect NVIDIA CUDA GPU first
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print(f"Using NVIDIA CUDA GPU: {torch.cuda.get_device_name(0)}")
        print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.2f} GB")
        return device

    # If no NVIDIA CUDA GPU, try to detect DirectML GPU
    try:
        if torch_directml.is_available():
            device = torch_directml.device()
            print(f"Using DirectML GPU: {device}")
            # Add a small test to ensure it's truly usable
            try:
                _ = torch.tensor([1], device=device)
            except Exception as e:
                print(f"Warning: DirectML device found but not usable ({e}). Falling back to CPU.")
                return torch.device("cpu")
            return device
        else:
            print("DirectML is NOT available.")
    except ImportError:
        print("torch_directml not installed.")
    except Exception as e:
        print(f"Error checking DirectML: {e}. Falling back to CPU.")

    # If neither GPU is found, fall back to CPU
    device = torch.device("cpu")
    print("No GPU (NVIDIA CUDA or DirectML) found. Using CPU.")
    return device


device = get_device()
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

In [ ]:
# print("\n--- Debugging df_tst Preparation ---")
# print(f"df_tst shape after fill_empty_examples_pandas: {df_tst_orig.shape}")
# print(f"df_tst 'inputs' head:\n{df_tst_orig['inputs'].head()}")
# # Check if any 'text_to_classify' is empty in test set
# df_tst_temp_dataset = MultiInputDataset(df_tst_orig, tokenizer, is_test=True)
# sample_item = df_tst_temp_dataset[0] # Get first item
# print(f"Sample test item keys: {sample_item.keys()}")
# print(f"Sample test item 'text_to_classify_input_ids' shape: {sample_item['text_to_classify_input_ids'].shape}")
# print(f"Sample test item 'rule_input_ids' shape: {sample_item['rule_input_ids'].shape}")
# print(f"Sample test item 'subreddit_input_ids' shape: {sample_item['subreddit_input_ids'].shape}")
# # Check if any of the input_ids are all zeros (indicating empty text)
# print(f"Test body input_ids sum (first item): {sample_item['text_to_classify_input_ids'].sum()}")
# print(f"Test rule input_ids sum (first item): {sample_item['rule_input_ids'].sum()}")
# print(f"Test subreddit input_ids sum (first item): {sample_item['subreddit_input_ids'].sum()}")

In [9]:
oof_preds = np.zeros(len(df_trn_orig))
test_preds_folds = [] # This is correct

for fold, (train_idx_orig, val_idx_orig) in enumerate(skf.split(df_trn_orig, df_trn_orig["rule_violation"])):
    print(f"\n----- Fold {fold+1} -----")
    # train_df = df_trn.iloc[train_idx].reset_index(drop=True)
    # 1. Create original train and validation DataFrames for this fold
    fold_train_df_orig = df_trn_orig.iloc[train_idx_orig].reset_index(drop=True)
    fold_val_df_orig = df_trn_orig.iloc[val_idx_orig].reset_index(drop=True)

    # 2. EXPAND the TRAINING data for this fold (ONLY from fold_train_df_orig)
    # This is the df_expanded_train for THIS fold
    expanded_train_data_list = []
    for idx, row in fold_train_df_orig.iterrows():
        # Add original body as a training sample
        expanded_train_data_list.append(create_multi_input_row(row, "body"))
        # Add positive examples
        expanded_train_data_list.append(create_multi_input_row(row, "positive_example_1"))
        expanded_train_data_list.append(create_multi_input_row(row, "positive_example_2"))
        # Add negative examples
        expanded_train_data_list.append(create_multi_input_row(row, "negative_example_1"))
        expanded_train_data_list.append(create_multi_input_row(row, "negative_example_2"))

    fold_train_df_expanded = pd.DataFrame(expanded_train_data_list)

    # Filter out rows where 'text_to_classify' might be empty after getText (from original NaNs)
    fold_train_df_expanded = fold_train_df_expanded[fold_train_df_expanded['text_to_classify'] != ''].reset_index(drop=True)

    # 3. Prepare the VALIDATION data for this fold (using original body from fold_val_df_orig)
    # This DataFrame will have an 'inputs' column with the 7 fields, where 'text_to_classify' is 'body'
    fold_val_df_for_model = fold_val_df_orig.copy()
    fold_val_df_for_model['text_to_classify'] = fold_val_df_for_model['body']
    # Ensure all 7 fields are present as columns
    fold_val_df_for_model['pos1'] = fold_val_df_for_model['positive_example_1'].apply(getText)
    fold_val_df_for_model['pos2'] = fold_val_df_for_model['positive_example_2'].apply(getText)
    fold_val_df_for_model['neg1'] = fold_val_df_for_model['negative_example_1'].apply(getText)
    fold_val_df_for_model['neg2'] = fold_val_df_for_model['negative_example_2'].apply(getText)
    fold_val_df_for_model['rule_violation'] = fold_val_df_orig['rule_violation'].values
    # Apply the fill logic for examples to fold_val_df_for_model as well
    fold_val_df_for_model = fill_empty_examples_pandas(fold_val_df_for_model)

    # 4. Create Datasets and DataLoaders
    train_dataset = MultiInputDataset(fold_train_df_expanded, tokenizer, 1e-6) # Train on expanded data
    val_dataset = MultiInputDataset(fold_val_df_for_model, tokenizer, 512) # Validate on original body

    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

    # Test loader is created once per fold, using df_tst_orig
    df_tst_for_model = df_tst_orig.copy()
    df_tst_for_model['text_to_classify'] = df_tst_for_model['body']
    # Ensure all 7 fields are present as columns
    df_tst_for_model['pos1'] = df_tst_for_model['positive_example_1'].apply(getText)
    df_tst_for_model['pos2'] = df_tst_for_model['positive_example_2'].apply(getText)
    df_tst_for_model['neg1'] = df_tst_for_model['negative_example_1'].apply(getText)
    df_tst_for_model['neg2'] = df_tst_for_model['negative_example_2'].apply(getText)
    # Apply the fill logic for examples to df_tst_for_model as well
    df_tst_for_model = fill_empty_examples_pandas(df_tst_for_model)

    test_loader = DataLoader(MultiInputDataset(df_tst_for_model, tokenizer, is_test=True), batch_size=8, shuffle=False)

    model = MultiInputBERT(model_name=MODEL_PATH).to(device)
    optimizer = AdamW(model.parameters(), lr=1e-6)
    criterion = nn.BCEWithLogitsLoss()

    num_training_steps_per_fold = len(train_loader) * N_EPOCHS
    num_warmup_steps_per_fold = int(num_training_steps_per_fold * 0.05) # 5% warmup

    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=num_warmup_steps_per_fold,
        num_training_steps=num_training_steps_per_fold
    )

    # Training Loop for this fold
    best_auc = -1.0
    best_model_state = None

    for epoch in range(N_EPOCHS):
        model.train()
        total_loss = 0
        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}"):
            inputs = {k: v.to(device) for k, v in batch.items() if k != "label"}
            labels = batch["label"].to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            logits = outputs.squeeze(-1)

            loss = criterion(logits, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0) # Gradient clipping
            optimizer.step()
            scheduler.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1} Loss: {total_loss / len(train_loader):.4f}")

        # Eval
        model.eval()
        all_val_preds_raw, all_val_labels = [], []
        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Validating Epoch {epoch + 1}"):
                inputs = {k: v.to(device) for k, v in batch.items() if k != "label"}
                labels = batch["label"].to(device)
                outputs = model(inputs)
                logits = outputs.squeeze(-1)

                probs = torch.sigmoid(logits).detach().cpu().tolist()
                all_val_preds_raw.extend(probs)
                all_val_labels.extend(labels.cpu().tolist())

            preds = [int(p > 0.5) for p in all_val_preds_raw]
            print(classification_report(all_val_labels, preds, digits=3, zero_division=0))
            curr_auc = roc_auc_score(all_val_labels, all_val_preds_raw)
            print(f"AUC Score: {curr_auc:.4f}")

            if curr_auc > best_auc:
                best_auc = curr_auc
                best_model_state = model.state_dict()
                print(f"  -> New best Val AUC for Fold {fold + 1}: {best_auc:.4f}")

    model.load_state_dict(best_model_state)
    print(f"Fold {fold + 1} Best Val AUC: {best_auc:.4f}")

    # Make OOF predictions for this fold's validation set
    model.eval()
    fold_val_preds_list = []
    fold_val_true_list = []
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Fold {fold + 1} OOF Prediction"):
            inputs = {k: v.to(device) for k, v in batch.items() if k != "label"}
            labels = batch["label"].to(device)
            outputs = model(inputs)
            logits = outputs.squeeze(-1)
            probs = torch.sigmoid(logits).detach().cpu().tolist()
            fold_val_preds_list.extend(probs)
            fold_val_true_list.extend(labels.cpu().tolist())

    oof_fold_auc_check = roc_auc_score(fold_val_true_list, fold_val_preds_list)
    print(f"Fold {fold + 1} OOF AUC Check: {oof_fold_auc_check:.4f} (Must match Best Val AUC)")

    oof_preds[val_idx_orig] = np.array(fold_val_preds_list) # Use val_idx_orig from kf.split

    # Make predictions on the TEST set using this fold's best model
    test_fold_preds = []
    with torch.no_grad():
        for batch in tqdm(test_loader, desc=f"Fold {fold + 1} Test Prediction"):
            inputs = {k: v.to(device) for k, v in batch.items()}
            outputs = model(inputs)
            logits = outputs.squeeze(-1)
            probs = torch.sigmoid(logits).detach().cpu().tolist()
            test_fold_preds.extend(probs)

    test_preds_folds.append(test_fold_preds)


----- Fold 1 -----


ValueError: Length of values (1623) does not match length of index (8115)

In [ ]:
# -----------------------------
# Final Calculation and Submission
# -----------------------------
overall_oof_auc = roc_auc_score(df_trn_orig['rule_violation'], oof_preds)
print(f"\n--- Overall {k_folds}-Fold OOF AUC: {overall_oof_auc:.4f} ---")

# Average test predictions across all folds
final_test_predictions = np.mean(test_preds_folds, axis=0)

print("\n--- Final Test Predictions Statistics ---")
print(f"Min: {np.min(final_test_predictions):.4f}")
print(f"Max: {np.max(final_test_predictions):.4f}")
print(f"Mean: {np.mean(final_test_predictions):.4f}")
print(f"Std Dev: {np.std(final_test_predictions):.4f}")
# Check distribution

import matplotlib
from matplotlib import pyplot as plt

plt.hist(final_test_predictions, bins=50)
plt.title("Distribution of Final Test Predictions")
plt.xlabel("Predicted Probability")
plt.ylabel("Count")
plt.show()

# Create final submission file
submission = pd.DataFrame({
    "row_id": df_tst_orig["row_id"],
    "rule_violation": final_test_predictions
})
submission.to_csv("submission.csv", index=False) # Save with a distinct name
print("K-Fold multi-input submission.csv created successfully!")
print(submission.head(10))